<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-maf-gradio/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install agent-framework gradio openai nest_asyncio --pre

In [ ]:
import nest_asyncio
import os
import gradio as gr
import asyncio
from agent_framework.openai import OpenAIChatClient
from google.colab import userdata
import warnings

# Patch the event loop for Jupyter compatibility
nest_asyncio.apply()

# Filter specific DeprecationWarning from jupyter_client
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

# Initialize API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
# 1. Define a tool
async def calculate_force(mass: float, acceleration: float):
    """Calculates force given mass (kg) and acceleration (m/s^2)."""
    return f"The calculated force is {mass * acceleration} Newtons."

# 2. Initialize the Client and Agent
client = OpenAIChatClient(model_id="gpt-3.5-turbo") # Automatically reads from OPENAI_API_KEY env var

agent = client.create_agent(
    name="PhysicsBot",
    instructions="You are a helpful physics assistant. Use tools for calculations.",
    tools=[calculate_force]
)

# 3. Define the Gradio Chat logic
async def chat_with_agent(message, history):
    # agent.run() is an async call
    response = await agent.run(message)
    return response.text

In [ ]:
# 4. Launch Gradio
# share=True is recommended for a stable external link
demo = gr.ChatInterface(
    fn=chat_with_agent,
    title="MAF Physics Assistant (Colab Edition)",
    description="Ask physics questions or use the Force Calculator tool!"
)

demo.launch(share=True, debug=True)